## Das Module multiprocessing

Erstellen Sie zwei parallelisierte Implementierung ihrer $\pi$-Bistimmung nach Monte Carlo, wovon eine auf *map* und eine auf *Queues* basiert. Vergleichen Sie die Rechenzeiten mit denen ihrer bisherigen Version. Wurden Ihre Erwartungen bezüglich der Beschleunigung erfüllt?

In [1]:
%load_ext cython

In [19]:
%%cython -a
from libc.stdlib cimport rand, srand, RAND_MAX
cimport cython

cpdef monte_carlo(quantity, seed):
    return cdef_monte_carlo(quantity, seed)

@cython.cdivision(True)
cdef int cdef_monte_carlo(int quantity, int seed):
    cdef int hits = 0
    cdef double width = 0.0
    cdef double height = 0.0
    cdef double temp = 0.0
    srand(seed)
    
    for _ in range(quantity):
        width = <double>rand()/RAND_MAX
        height = <double>rand()/RAND_MAX
        if width * width + height * height < 1:
            hits += 1

    return hits

In [37]:
#Monte Carlo Test
import random

random_number = int(random.random() * (2**31 -1))
trys = 25_000_000
print(monte_carlo(trys, random_number) / trys * 4)

3.14124992


In [38]:
%%timeit
monte_carlo(10_000, random_number)

255 µs ± 448 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
def worker_monte_carlo(q_in, q_out):
    while True:
        arguments = q_in.get()
        x, y = arguments
        result = monte_carlo(x,y)
        q_out.put(result)
        q_in.task_done()

In [22]:
import multiprocessing
import math
import random
nrOfCores = multiprocessing.cpu_count()
print('nrOfCores:', nrOfCores)

l = []
for i in range (128):
    random_number = int(random.random() * (2**31 -1))
    l.append((2_500_000, random_number))

nrOfCores: 8


In [23]:
print(l)

[(2500000, 548878320), (2500000, 460053794), (2500000, 443731170), (2500000, 2134945269), (2500000, 2041407812), (2500000, 1566822710), (2500000, 331130664), (2500000, 304192205), (2500000, 463688621), (2500000, 157905095), (2500000, 2004265644), (2500000, 1929364814), (2500000, 1945549326), (2500000, 2138981631), (2500000, 1587969233), (2500000, 1625245269), (2500000, 1798815847), (2500000, 1997551104), (2500000, 836520581), (2500000, 1674788092), (2500000, 998537626), (2500000, 2097048545), (2500000, 1250626911), (2500000, 600337875), (2500000, 1678504531), (2500000, 1970745930), (2500000, 1387240631), (2500000, 42322506), (2500000, 1076658937), (2500000, 1371031838), (2500000, 221631278), (2500000, 1433570724), (2500000, 487519170), (2500000, 947771891), (2500000, 1493108286), (2500000, 1847718968), (2500000, 766575470), (2500000, 1544078241), (2500000, 987770294), (2500000, 808381113), (2500000, 2031905443), (2500000, 1760747215), (2500000, 1598272442), (2500000, 546312777), (25000

In [24]:
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_monte_carlo, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

print(result_list)

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result_tuple in result_list:
    hits += result_tuple

    
print((hits)/trys * 4)
for p in processes:
    p.terminate()

[1963713, 1963713, 1963713, 1963713, 1963713, 1963713, 1963713, 1962316, 1963713, 1962316, 1962551, 1962316, 1962316, 1962551, 1962097, 1962316, 1962551, 1962097, 1962316, 1962551, 1962097, 1962840, 1962551, 1962097, 1962840, 1962316, 1962551, 1962551, 1962097, 1962840, 1962840, 1962840, 1963137, 1963137, 1963547, 1962097, 1962840, 1963137, 1963547, 1963137, 1963547, 1962935, 1963137, 1963547, 1963137, 1963547, 1962935, 1964120, 1962316, 1962551, 1962097, 1962840, 1963137, 1963547, 1962935, 1964120, 1963659, 1964120, 1963659, 1962935, 1964120, 1963659, 1963676, 1962935, 1963676, 1963437, 1962097, 1963547, 1962935, 1964120, 1963659, 1963676, 1963437, 1964334, 1963659, 1963676, 1963437, 1964334, 1962935, 1964334, 1963402, 1963610, 1962840, 1963137, 1963547, 1962935, 1964120, 1963659, 1963676, 1963437, 1964334, 1963402, 1963610, 1964120, 1963676, 1963402, 1964120, 1963402, 1963610, 1963312, 1963312, 1963659, 1963676, 1963437, 1963659, 1963676, 1963437, 1963610, 1963437, 1963312, 1963312, 

## TimeIt

In [25]:
%%timeit

import multiprocessing
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_monte_carlo, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

print(result_list)

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print((hits)/trys * 4)
for p in processes:
    p.terminate()
    
#5.96 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#6.03 s ± 70.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.9 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

#without hypot
#1.94 s ± 5.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

[1963713, 1963713, 1963713, 1963713, 1962316, 1963713, 1962316, 1962316, 1963713, 1962316, 1962316, 1962316, 1963713, 1962316, 1963713, 1962316, 1962551, 1962551, 1962551, 1962551, 1962551, 1962551, 1962551, 1962551, 1962097, 1962097, 1962097, 1962097, 1962097, 1962097, 1962097, 1962097, 1962840, 1962840, 1962840, 1962840, 1962840, 1962840, 1962840, 1962840, 1963137, 1963137, 1963137, 1963137, 1963137, 1963137, 1963137, 1963137, 1963547, 1963547, 1963547, 1963547, 1963547, 1963547, 1963547, 1963547, 1962935, 1962935, 1962935, 1962935, 1962935, 1962935, 1962935, 1962935, 1964120, 1964120, 1964120, 1964120, 1964120, 1964120, 1964120, 1964120, 1963659, 1963659, 1963659, 1963659, 1963659, 1963659, 1963659, 1963659, 1963676, 1963676, 1963676, 1963676, 1963676, 1963676, 1963676, 1963676, 1963437, 1963437, 1963437, 1963437, 1963437, 1963437, 1963437, 1963437, 1964334, 1964334, 1964334, 1964334, 1964334, 1964334, 1964334, 1964334, 1963402, 1963402, 1963402, 1963402, 1963402, 1963402, 1963402, 

## Map

In [ ]:
%%timeit
x = [e[0] for e in l]
y = [e[1] for e in l] 
result_list = map(monte_carlo, x, y)

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print(result_list)
print((hits)/trys * 4)

#23.9 s ± 205 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#23.5 s ± 86.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%timeit
process_pool = multiprocessing.Pool(processes = nrOfCores)
result_list = process_pool.starmap(monte_carlo, l)
process_pool.close()

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print(result_list)
print((hits)/trys * 4)

#5.88 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 49.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.74 s ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## Numba JIT Test

In [52]:
from numba import jit
import random

@jit
def monte_carlo(quantity, seed):
    random.seed(seed)
    hits = 0
    width = 0.0
    height = 0.0
    temp = 0.0
    
    for _ in range(quantity):
        width = random.random()
        height = random.random()
        if width * width + height * height < 1:
            hits += 1

    return hits

In [51]:
trys = 2_500_000
print(monte_carlo(trys, 5123432) / trys * 4)

3.1412768


In [58]:
#%%timeit

import multiprocessing
import math
import random
nrOfCores = multiprocessing.cpu_count()
print('nrOfCores:', nrOfCores)

l = []
for i in range (128):
    random_number = int(random.random() * (2**31 -1))
    l.append((250_000_000, random_number))

process_pool = multiprocessing.Pool(processes = nrOfCores)
result_list = process_pool.starmap(monte_carlo, l)
process_pool.close()

hits = 0
trys = 0

for e in l:
    x, y = e
    trys += x

for result in result_list:
    hits += result
    
print(result_list)
print((hits)/trys * 4)

#5.88 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 49.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.74 s ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

nrOfCores: 8
[196347968, 196347336, 196351694, 196353422, 196360791, 196347000, 196351264, 196342595, 196352216, 196356821, 196362915, 196345210, 196353919, 196355073, 196348013, 196353807, 196355024, 196347590, 196364789, 196353188, 196347278, 196348456, 196347793, 196344541, 196339164, 196348852, 196349647, 196342749, 196339601, 196346728, 196365575, 196345626, 196349823, 196343391, 196335068, 196350163, 196337766, 196353645, 196355224, 196345136, 196347448, 196341826, 196345754, 196348553, 196360833, 196352658, 196352635, 196347947, 196338137, 196356567, 196354239, 196362806, 196353094, 196343731, 196350787, 196352656, 196352750, 196360114, 196358947, 196358258, 196358249, 196332579, 196349181, 196349499, 196347999, 196358138, 196344585, 196365358, 196351363, 196352129, 196345412, 196343411, 196356210, 196345112, 196356213, 196347963, 196344749, 196337966, 196348720, 196345055, 196348638, 196355965, 196351405, 196350346, 196338495, 196355674, 196345904, 196364211, 196352846, 1963477